In [7]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [9]:
def get_cuurent_weather(city, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'  # pour obtenir les données en degrés Celsius
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'description': data['weather'][0]['description'],
            'humidity': data['main']['humidity'],
            'pressure': data['main']['pressure'],
            'wind_speed': data['wind']['speed']
        }
        return weather
    else:
        return None

In [13]:
# 'your_api_key' par votre propre clé API obtenue de OpenWeatherMap
api_key = '40ff5ede1101ddc6e4a3f0f819eded84'
city = 'Paris'
#base_url = 'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'
#response = requests.get(base_url, params=params)
weather = get_cuurent_weather(city, api_key)

if weather:
    print(f"Weather in {weather['city']}:")
    print(f"Temperature: {weather['temperature']}°C")
    print(f"Description: {weather['description']}")
    print(f"Humidity: {weather['humidity']}%")
    print(f"Pressure: {weather['pressure']} hPa")
    print(f"Wind Speed: {weather['wind_speed']} m/s")
else:
    print("Failed to get weather data", response.status_code)

Weather in Paris:
Temperature: 19.29°C
Description: overcast clouds
Humidity: 74%
Pressure: 1016 hPa
Wind Speed: 5.14 m/s


## II. Prévisions météo à 5 jours


In [15]:
def previsions_meteo(city, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric', # pour obtenir les données en degrés Celsius
        'cnt': 40
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        print(data)
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'temp_min': data['temperature']['min'],
            'temp_max':data['temperature']['max'],
            'weather' : data['weather']['value'],
            'humidity': data['main']['humidity'],
            'pressure': data['main']['pressure'],
            #'wind_speed': data['wind']['speed'], 
            #'local_time': data['city']['timezone']
            'weather': data['weather'][0]['description'],
            'wind_speed': data['wind']['speed'],
            'local_time': datetime.utcfromtimestamp(data['dt']) + timedelta(seconds=data['city']['timezone'])
        }
        return weather
    else:
        return None

Les éléments à extraire comprennent des détails sur les conditions météorologiques à différents moments :

city : Nom de la ville pour laquelle les prévisions sont fournies.
temperature : Température actuelle en degrés Celsius.
temp_min : Température minimale prévue.
temp_max : Température maximale prévue.
pressure : Pression atmosphérique en hPa (hectopascals).
humidity : Humidité relative en pourcentage.
weather : Description textuelle des conditions météorologiques (par exemple, "ciel dégagé", "pluie légère").
wind_speed : Vitesse du vent en mètres par seconde.
local_time : Heure locale à laquelle la prévision s'applique, convertie à partir de l'heure UTC et ajustée au fuseau horaire de la ville.

In [21]:
def previsions_meteo(city, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/forecast"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric',  # pour obtenir les données en degrés Celsius
        'cnt': 40  # nombre maximal de prévisions à récupérer (5 jours à raison de 8 prévisions par jour)
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        #print(data)
        
        forecasts = []
        
        for dat in data['list']:
            forecast = {
                'city': data['city']['name'],
                'temperature': dat['main']['temp'],
                'temp_min': dat['main']['temp_min'],
                'temp_max': dat['main']['temp_max'],
                'pressure': dat['main']['pressure'],
                'humidity': dat['main']['humidity'],
                'weather': dat['weather'][0]['description'],
                '   wind_speed': dat['wind']['speed'],
                'local_time     ': datetime.utcfromtimestamp(dat['dt']) + timedelta(seconds=data['city']['timezone'])
            }
            forecasts.append(forecast)
        
        return forecasts
    else:
        return None

In [25]:
#This function iterates over a list of cities, calls previsions_meteo for each city, and compiles weather forecast data for all cities.
def get_forecast_data(cities, api_key):
    all_forecasts = []
    for city in cities:
        city_forecasts = previsions_meteo(city, api_key)
        if city_forecasts:
            all_forecasts.extend(city_forecasts)
        else:
            print(f"Failed to get weather data for {city}")
    return all_forecasts

def convert_to_dataframe(forecast_data):
    df = pd.DataFrame(forecast_data)
    return df

api_key = '40ff5ede1101ddc6e4a3f0f819eded84'
cities = ['London', 'New York', 'Paris']

forecast_data = get_forecast_data(cities, api_key)
df_forecast = convert_to_dataframe(forecast_data)

print(df_forecast.head())

     city  temperature  temp_min  temp_max  pressure  humidity  \
0  London        16.45     16.45     17.02      1013        86   
1  London        15.31     14.89     15.31      1015        82   
2  London        13.72     13.72     13.72      1017        88   
3  London        13.02     13.02     13.02      1017        90   
4  London        12.96     12.96     12.96      1017        88   

           weather     wind_speed     local_time       
0  overcast clouds           2.75 2024-10-25 16:00:00  
1       light rain           1.80 2024-10-25 19:00:00  
2  overcast clouds           1.33 2024-10-25 22:00:00  
3  overcast clouds           1.12 2024-10-26 01:00:00  
4    broken clouds           1.78 2024-10-26 04:00:00  


In [139]:
df_forecast

,city,temperature,temp_min,temp_max,pressure,humidity,weather,wind_speed,local_time
0,London,18.56,18.56,18.94,1007,73,light rain,4.77,2024-07-16 13:00:00
1,London,18.26,18.21,18.26,1010,77,light rain,4.42,2024-07-16 16:00:00
2,London,19.84,19.84,19.84,1012,68,light rain,3.45,2024-07-16 19:00:00
3,London,17.18,17.18,17.18,1015,86,light rain,2.31,2024-07-16 22:00:00
4,London,15.20,15.20,15.20,1016,95,light rain,1.93,2024-07-17 01:00:00
...,...,...,...,...,...,...,...,...,...
115,Paris,22.59,22.59,22.59,1015,65,scattered clouds,3.38,2024-07-20 23:00:00
116,Paris,20.72,20.72,20.72,1016,59,broken clouds,2.03,2024-07-21 02:00:00
117,Paris,18.87,18.87,18.87,1016,61,overcast clouds,1.56,2024-07-21 05:00:00
118,Paris,19.07,19.07,19.07,1016,59,overcast clouds,2.27,2024-07-21 08:00:00


## III. Données de pollution de l'air actuelles

Vous devez récupérer les données actuelles de pollution de l'air pour la ville de Paris ainsi que pour deux autres villes de votre choix.

Les éléments à extraire comprennent des détails sur les niveaux de différents polluants à un moment donné :

city : Nom de la ville pour laquelle les données de pollution sont fournies.
aqi : Indice de qualité de l'air (Air Quality Index).
co : Concentration de monoxyde de carbone (CO) en µg/m³.
no : Concentration de monoxyde d'azote (NO) en µg/m³.
no2 : Concentration de dioxyde d'azote (NO2) en µg/m³.
o3 : Concentration d'ozone (O3) en µg/m³.
so2 : Concentration de dioxyde de soufre (SO2) en µg/m³.
pm2_5 : Concentration de particules fines (PM2.5) en µg/m³.
pm10 : Concentration de particules grossières (PM10) en µg/m³.
nh3 : Concentration d'ammoniac (NH3) en µg/m³.
local_time : Heure locale à laquelle les données de pollution s'appliquent, 

In [144]:
from datetime import datetime, timedelta

def fetch_air_pollution_data(city, api_key):
    geocode_url = "http://api.openweathermap.org/geo/1.0/direct"
    geocode_params = {
        'q': city,
        'appid': api_key,
        'limit': 1
    }
    
    geocode_response = requests.get(geocode_url, params=geocode_params)
    if geocode_response.status_code == 200:
        geocode_data = geocode_response.json()
        if not geocode_data:
            return None
        lat = geocode_data[0]['lat']
        lon = geocode_data[0]['lon']
    else:
        return None

    # Now fetch the air pollution data using the coordinates
    air_pollution_url = "http://api.openweathermap.org/data/2.5/air_pollution"
    air_pollution_params = {
        'lat': lat,
        'lon': lon,
        'appid': api_key
    }

    air_pollution_response = requests.get(air_pollution_url, params=air_pollution_params)
    if air_pollution_response.status_code == 200:
        air_data = air_pollution_response.json()
        print(air_data)
        pollution = air_data['list'][0]
        timestamp = pollution['dt']
        local_time = datetime.utcfromtimestamp(timestamp) + timedelta(seconds=geocode_data[0].get('timezone', 0))

        air_pollution = {
            'city': city,
            'aqi': pollution['main']['aqi'],
            'co': pollution['components']['co'],
            'no': pollution['components']['no'],
            'no2': pollution['components']['no2'],
            'o3': pollution['components']['o3'],
            'so2': pollution['components']['so2'],
            'pm2_5': pollution['components']['pm2_5'],
            'pm10': pollution['components']['pm10'],
            'nh3': pollution['components']['nh3'],
            'local_time': local_time
        }
        return air_pollution
    else:
        return None
        

In [146]:
def get_pollution_data(cities, api_key):
    all_pollution_data = []
    for city in cities:
        city_pollution_data = fetch_air_pollution_data(city, api_key)
        if city_pollution_data:
            all_pollution_data.append(city_pollution_data)
        else:
            print(f"Failed to get air pollution data for {city}")
    return all_pollution_data

def convert_to_dataframe(pollution_data):
    df = pd.DataFrame(pollution_data)
    return df

api_key = 'your_api'  
cities = ['Saint-Petersbourg', 'London', 'New York', 'Moscow', 'Berlin', 'Kolkata']

pollution_data = get_pollution_data(cities, api_key)
df_pollution = convert_to_dataframe(pollution_data)

print(df_pollution.head())

{'coord': {'lon': 30.3162, 'lat': 59.9387}, 'list': [{'main': {'aqi': 3}, 'components': {'co': 185.25, 'no': 1.2, 'no2': 6.17, 'o3': 115.87, 'so2': 9.89, 'pm2_5': 3.82, 'pm10': 4.43, 'nh3': 0.35}, 'dt': 1721122437}]}
{'coord': {'lon': -0.1276, 'lat': 51.5073}, 'list': [{'main': {'aqi': 2}, 'components': {'co': 158.55, 'no': 0.57, 'no2': 7.45, 'o3': 63.66, 'so2': 2.92, 'pm2_5': 2.23, 'pm10': 2.36, 'nh3': 0.45}, 'dt': 1721122924}]}
{'coord': {'lon': -74.006, 'lat': 40.7127}, 'list': [{'main': {'aqi': 1}, 'components': {'co': 260.35, 'no': 0.01, 'no2': 12.85, 'o3': 30.76, 'so2': 2.98, 'pm2_5': 4.99, 'pm10': 6.19, 'nh3': 0.38}, 'dt': 1721122924}]}
{'coord': {'lon': 37.6175, 'lat': 55.7504}, 'list': [{'main': {'aqi': 3}, 'components': {'co': 433.92, 'no': 38.89, 'no2': 123.38, 'o3': 49.35, 'so2': 66.76, 'pm2_5': 49.06, 'pm10': 55.35, 'nh3': 0}, 'dt': 1721122457}]}
{'coord': {'lon': 13.3889, 'lat': 52.517}, 'list': [{'main': {'aqi': 2}, 'components': {'co': 233.65, 'no': 1.01, 'no2': 14.22, 

In [147]:
df_pollution

,city,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,local_time
0,Saint-Petersbourg,3,185.25,1.20,6.17,115.87,9.89,3.82,4.43,0.35,2024-07-16 09:33:57
1,London,2,158.55,0.57,7.45,63.66,2.92,2.23,2.36,0.45,2024-07-16 09:42:04
2,New York,1,260.35,0.01,12.85,30.76,2.98,4.99,6.19,0.38,2024-07-16 09:42:04
3,Moscow,3,433.92,38.89,123.38,49.35,66.76,49.06,55.35,0.00,2024-07-16 09:34:17
4,Berlin,2,233.65,1.01,14.22,81.54,5.31,5.95,6.42,0.71,2024-07-16 09:42:04
5,Kolkata,2,520.71,0.88,24.68,23.60,34.33,16.71,21.84,6.71,2024-07-16 09:42:04


 ## IV. Prévisions de pollution de l'air à 4 jours


In [33]:
def fetch_air_pollution_forecast(city, api_key):
    geocode_url = "http://api.openweathermap.org/geo/1.0/direct"
    geocode_params = {
        'q': city,
        'appid': api_key,
        'limit': 1
    }
    
    geocode_response = requests.get(geocode_url, params=geocode_params)
    if geocode_response.status_code == 200:
        geocode_data = geocode_response.json()
        if not geocode_data:
            return None
        lat = geocode_data[0]['lat']   
        lon = geocode_data[0]['lon']   
    else:
        return None

    air_pollution_forecast_url = "http://api.openweathermap.org/data/2.5/air_pollution/forecast"
    air_pollution_forecast_params = {
        'lat': lat,
        'lon': lon,
        'appid': api_key
    }

    air_pollution_forecast_response = requests.get(air_pollution_forecast_url, params=air_pollution_forecast_params)
    if air_pollution_forecast_response.status_code == 200:
        forecast_data = air_pollution_forecast_response.json() 
        forecast_list = forecast_data['list'] 

        
        forecast_details = []#list dict
        for forecast in forecast_list:
            timestamp = forecast['dt']
            local_time = datetime.utcfromtimestamp(timestamp) + timedelta(seconds=geocode_data[0].get('timezone', 0))

            
            air_pollution = {
                'city': city,
                'aqi': forecast['main']['aqi'],
                'co': forecast['components']['co'],
                'no': forecast['components']['no'],
                'no2': forecast['components']['no2'],
                'o3': forecast['components']['o3'],
                'so2': forecast['components']['so2'],
                'pm2_5': forecast['components']['pm2_5'],
                'pm10': forecast['components']['pm10'],
                'nh3': forecast['components']['nh3'],
                'local_time': local_time
            }
            forecast_details.append(air_pollution)
        return forecast_details
    else:
        return None

def get_forecast_data(cities, api_key):
    all_forecast_data = []
    for city in cities:
        city_forecast_data = fetch_air_pollution_forecast(city, api_key)
        if city_forecast_data:
            all_forecast_data.extend(city_forecast_data)
        else:
            print(f'Failed to get air pollution forecast data for {city}')
    return all_forecast_data        


api_key = '40ff5ede1101ddc6e4a3f0f819eded84'  
cities = ['Saint-Petersbourg', 'London', 'New York', 'Moscow', 'Berlin', 'Kazan']

forecast_data = get_forecast_data(cities, api_key)
df_forecast = convert_to_dataframe(forecast_data)

print(df_forecast.head())

                city  aqi      co    no    no2     o3    so2  pm2_5  pm10  \
0  Saint-Petersbourg    2  347.14  5.92  30.16   7.60  33.85   6.22  9.76   
1  Saint-Petersbourg    2  300.41  1.50  26.05  19.31  23.13   4.93  7.88   
2  Saint-Petersbourg    2  297.07  0.26  23.65  28.97  20.98   3.18  5.02   
3  Saint-Petersbourg    1  297.07  0.05  21.08  36.12  19.31   2.36  3.61   
4  Saint-Petersbourg    1  300.41  0.07  22.96  32.90  19.79   2.62  4.02   

    nh3          local_time  
0  0.17 2024-10-25 12:00:00  
1  0.15 2024-10-25 13:00:00  
2  0.15 2024-10-25 14:00:00  
3  0.19 2024-10-25 15:00:00  
4  0.21 2024-10-25 16:00:00  


In [35]:
df_forecast

,city,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,local_time
0,Saint-Petersbourg,2,347.14,5.92,30.16,7.60,33.85,6.22,9.76,0.17,2024-10-25 12:00:00
1,Saint-Petersbourg,2,300.41,1.50,26.05,19.31,23.13,4.93,7.88,0.15,2024-10-25 13:00:00
2,Saint-Petersbourg,2,297.07,0.26,23.65,28.97,20.98,3.18,5.02,0.15,2024-10-25 14:00:00
3,Saint-Petersbourg,1,297.07,0.05,21.08,36.12,19.31,2.36,3.61,0.19,2024-10-25 15:00:00
4,Saint-Petersbourg,1,300.41,0.07,22.96,32.90,19.79,2.62,4.02,0.21,2024-10-25 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...
571,Kazan,1,247.00,0.73,7.54,26.11,3.37,4.56,5.03,0.11,2024-10-29 07:00:00
572,Kazan,1,247.00,0.85,7.20,26.11,3.19,5.09,5.56,0.09,2024-10-29 08:00:00
573,Kazan,1,243.66,0.64,6.68,28.61,2.86,5.24,5.68,0.07,2024-10-29 09:00:00
574,Kazan,1,240.33,0.31,5.91,33.26,2.38,4.91,5.28,0.05,2024-10-29 10:00:00
